# DECaLS predictions

Run the test set through the pretrained model

## Setup

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import h5py

Importing project modules works best from the project root directory.

In [70]:
os.chdir('..')

import zoobot
import projcode
from projcode.data.utils import read_params
from projcode.predictions.make_predictions import decals_catalog
from projcode.predictions.view_predictions import ViewPredictions
from zoobot.shared.label_metadata import decals_pairs

In [3]:
params = read_params()

## Check the catalog file

There are also shards in `tfrecord` format, but these are bizarrely difficult to work with. Better to keep it simple.

In [4]:
dataroot = Path(params['dataroot'])
catalog_file = dataroot / 'shards/decals/test_shards/test_df.csv'

In [5]:
df = pd.read_csv(catalog_file)
df.head(5)

,Unnamed: 0,id_str,file_loc,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,disk-edge-on_yes,disk-edge-on_no,has-spiral-arms_yes,has-spiral-arms_no,...,spiral-arm-count_1,spiral-arm-count_2,spiral-arm-count_3,spiral-arm-count_4,spiral-arm-count_more-than-4,spiral-arm-count_cant-tell,merging_none,merging_minor-disturbance,merging_major-disturbance,merging_merger
0,111075,J141907.77+042635.7,/home/colin/data/munch1tb/zoobot_data/gz_decal...,1.0,4.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,22533,J120551.27+242120.5,/home/colin/data/munch1tb/zoobot_data/gz_decal...,1.0,3.0,1.0,0.0,3.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0
2,124358,J001555.69+161458.0,/home/colin/data/munch1tb/zoobot_data/gz_decal...,4.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0
3,47608,J103628.30+000818.0,/home/colin/data/munch1tb/zoobot_data/gz_decal...,4.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0
4,90582,J082651.90+140856.1,/home/colin/data/munch1tb/zoobot_data/gz_decal...,3.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


## Run predictions

Ignore all the nonsense about NUMA node. Users have been complaining about this for years, but Tensorflow still does it.

In [51]:
save_loc = dataroot / 'results/predictions/decals.hdf5'

In [6]:
# %time decals_catalog(params, save_loc)

## View predictions

In [8]:
# with h5py.File(save_loc, "r") as f:
#     id_str = [id.decode("utf-8") for id in list(f['id_str'])]
#     label_cols = [l.decode("utf-8") for l in list(f['label_cols'])]
#     predictions = list(f['predictions'])

In [71]:
vp = ViewPredictions()
vp.read_predictions(save_loc)
vp.make_summaries()

In [77]:
vp.results.head(2)

,file_loc,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,disk-edge-on_yes,disk-edge-on_no,has-spiral-arms_yes,has-spiral-arms_no,bar_strong,bar_weak,...,spiral-arm-count_1,spiral-arm-count_2,spiral-arm-count_3,spiral-arm-count_4,spiral-arm-count_more-than-4,spiral-arm-count_cant-tell,merging_none,merging_minor-disturbance,merging_major-disturbance,merging_merger
0,/home/colin/data/munch1tb/zoobot_data/gz_decal...,24.585224,47.126320,2.547081,37.434212,2.384364,6.699317,4.103290,1.479639,5.054061,...,1.128513,9.593546,1.531305,2.105283,1.051178,48.488930,75.635941,6.258410,1.058971,1.219766
1,/home/colin/data/munch1tb/zoobot_data/gz_decal...,3.121219,43.051598,1.741715,1.676396,90.948959,35.482941,1.227375,1.143801,5.037024,...,1.000283,1.206696,1.258230,1.271014,1.000010,3.857238,40.318962,9.000361,2.562151,1.091530


In [78]:
vp.summaries.head(2)

,file_loc,smooth-or-featured,disk-edge-on,has-spiral-arms,bar,bulge-size,how-rounded,edge-on-bulge,spiral-winding,spiral-arm-count,merging
0,/home/colin/data/munch1tb/zoobot_data/gz_decal...,"[[smooth-or-featured, featured-or-disk], 0.634...","[[disk-edge-on, yes], 0.9401192665100098]","[[has-spiral-arms, yes], 0.6201574802398682]","[[bar, no], 0.6682100892066956]","[[bulge-size, small], 0.7810470461845398]","[[how-rounded, cigar-shaped], 0.9727775454521179]","[[edge-on-bulge, none], 0.6480322480201721]","[[spiral-winding, tight], 0.6849639415740967]","[[spiral-arm-count, cant-tell], 0.758839964866...","[[merging, none], 0.8985763788223267]"
1,/home/colin/data/munch1tb/zoobot_data/gz_decal...,"[[smooth-or-featured, featured-or-disk], 0.898...","[[disk-edge-on, no], 0.9819012880325317]","[[has-spiral-arms, yes], 0.9665659070014954]","[[bar, no], 0.7243795990943909]","[[bulge-size, small], 0.789142370223999]","[[how-rounded, in-between], 0.7172414660453796]","[[edge-on-bulge, rounded], 0.7362374663352966]","[[spiral-winding, tight], 0.7864148616790771]","[[spiral-arm-count, cant-tell], 0.402069002389...","[[merging, none], 0.7611227631568909]"
